In [8]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib
from flask_cors import CORS
from collections import Counter
from statistics import mean

loaded_pipeline = joblib.load('svm_model.joblib')

scaler = loaded_pipeline.named_steps['standardscaler']
svm_classifier = loaded_pipeline.named_steps['svc']

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def filteredPrediction(prediction, confidence_percentages):
    filtered_data = [(pred, confidence) for pred, confidence in zip(prediction, confidence_percentages) if confidence > 30]
    
    # Extract filtered predictions and confidences
    filtered_predictions, filtered_confidences = zip(*filtered_data)
    
    # Count occurrences of each prediction
    prediction_counts = Counter(filtered_predictions)
    
    # Determine the most frequent prediction
    most_frequent_prediction = prediction_counts.most_common(1)[0][0]
    
    # Calculate average confidence
    average_confidence = mean(filtered_confidences) if filtered_confidences else 0
    return most_frequent_prediction,average_confidence

app = Flask(__name__)
CORS(app)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'message': 'No file part'}), 400

    file = request.files['file']
    
    # Do something with the file, e.g., save it to the server
    file.save('uploaded_file.csv')

    return jsonify({'message': 'File uploaded successfully'}), 200

@app.route('/predict', methods=['POST'])
def predict():
    received_data = pd.read_csv('uploaded_file.csv')
    X_new = received_data
    X_new_standardized = scaler.transform(X_new)
    prediction = svm_classifier.predict(X_new_standardized).astype(int)
    
    confidence_scores = svm_classifier.decision_function(X_new_standardized)
    # print(type(confidence_scores))
    confidence_percentages = np.array([round(float(sigmoid(i) * 100), 2) for i in confidence_scores])
    finalPred,confidence_percentage = filteredPrediction(prediction, confidence_percentages)

    return jsonify({'prediction': finalPred.tolist(), 'confidence': confidence_percentage.tolist()})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [26/Mar/2024 12:48:53] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2024 12:49:00] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2024 12:49:01] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2024 12:49:30] "POST /predict HTTP/1.1" 200 -


In [7]:
arr=[ 66.75,  30.53,  43.53, 59.13, 65.13 ]
avg=mean(arr)
avg

53.014